# Transacciones

En la blockchain que vamos a utilizar en este seminario, se ha diseñado una transacción básica que contiene los siguientes datos:

- **Pagador:** Identificador del usuario que envía la transacción.
- **Receptor:** Identificador del usuario que recibe la transacción.
- **Cantidad:** Cantidad de dinero transferido.

In [14]:
from typing import Dict

# Clase para almacenar los datos de una transacción
class Transaccion:
    def __init__(self, pagador: str, receptor: str, cantidad: float):
        self.pagador = pagador
        self.receptor = receptor
        self.cantidad = cantidad

    # Devuelve los datos de la transacción en un diccionario
    def convert_to_dict(self) -> Dict:
        return {
            'pagador': self.pagador,
            'receptor': self.receptor,
            'cantidad': self.cantidad
        }

    # Convertimos a un string el contenido de la transacción
    def __str__(self) -> str:
        return f"{self.pagador} envia a {self.receptor} la cantidad de {self.cantidad}"


Creamos una transacción de ejemplo.

In [13]:
# Creamos una transacción
t1 = Transaccion("Pepe", "Juan", 20)

Utilizamos el método `convert_to_dict()` para convertir la transacción en un objetio de tipo diccionario.

In [12]:
# Convertimos la transacción a un diccionario
t1.convert_to_dict()

{'pagador': 'Pepe', 'receptor': 'Juan', 'cantidad': 20}

En el siguiente ejemplo vamos a ver la utilidad del método `__str__` cuando se imprima el objeto.

In [11]:
# Imprimimos el contenido de la transacción
print(t1)

Pepe envia a Juan la cantidad de 20


# Árbol de Merkle

Un árbol de Merkle es una estructura de datos en árbol en el que cada nodo que no es una hoja está etiquetado con el hash que resulta de concatenar los hashes de sus nodos hijo.

In [16]:
from typing import List

# Clase para implementar un árbol de Merkle que almacena las transacciones de un bloque.
# Mirar los apuntes de EDA.
class MerkleTree:
    def __init__(self, l_transacciones: List[Transaccion]):
        self.l_transacciones = l_transacciones
        self.root, self.niveles = self.construir_merkle_tree()

    def construir_merkle_tree(self) -> tuple[str, List[List[str]]]:
        if not self.l_transacciones:
            return "", []

        # Generamos los hashs de la lista de transacciones (Versión Iterativa)
        hojas = []
        for tx in self.l_transacciones:
            # (1) Convertimos la transacción en un diccionario: tx.convert_to_dict()
            # (2) Convertimos el diccionario en una cadena: str(tx.convert_to_dict())
            # (3) Convertimos la cadena a bytes: str(tx.to_dict()).encode()
            # (4) Creamos el hash de la transacción y obtenemos su resumen en hexadecimal: hashlib.sha256(...).hexdigest()
            hash_transaccion = hashlib.sha256(str(tx.convert_to_dict()).encode()).hexdigest()

            # Añadimos el hash a la lista de hojas
            hojas.append(hash_transaccion)

        # Guardamos los niveles del árbol de Merkle
        niveles = [hojas]

        # Si son más transacciones, calculamos los hash de cada una de ellas y construimos el árbol
        # Construimos el árbol, de abajo a arriba
        # Acordaos de lo que habéis visto en EDA
        nivel_actual = hojas
        while len(nivel_actual) > 1:
            nivel_temp = []
            for i in range(0, len(nivel_actual), 2):
                if i + 1 < len(nivel_actual):
                    # Juntamos dos hojas
                    combined = nivel_actual[i] + nivel_actual[i + 1]
                    nuevo_hash = hashlib.sha256(combined.encode()).hexdigest()
                    nivel_temp.append(nuevo_hash)
                else:
                    # Hoja impar primero
                    nivel_temp.append(nivel_actual[i])
            niveles.append(nivel_temp)
            nivel_actual = nivel_temp

        return nivel_actual[0], niveles

    def mostrar_niveles(self) -> None:
        print("\nNiveles del Árbol de Merkle:")
        for idx, nivel in enumerate(reversed(self.niveles)):
            print(f"\tNivel {idx} (desde la raíz):")
            for hash in nivel:
                print(f"\t\t{hash}")


Vamos a crear un árbol de Merkle de ejemplo con 4 transacciones.

In [19]:
# Creamos una lista con 4 transacciones de ejemplo
lista_transacciones = [Transaccion("Pepe", "Juan", 10), Transaccion("María", "Lucía", 20), Transaccion("Antonio", "Ramón",30), Transaccion("Carmen", "Marta", 40)]


Una vez que hemos creado la lista de transacciones ya podemos crear el árbol de Merkle.

In [22]:
import hashlib

# Creamos un objeto de tipo MerkleTree
arbol_merkle = MerkleTree(lista_transacciones)

Mostramos los niveles del árbol haciendo uso del método `mostrar_niveles`.

In [23]:
arbol_merkle.mostrar_niveles()


Niveles del Árbol de Merkle:
	Nivel 0 (desde la raíz):
		6d84b95380981201807f01b53207747bb2d1e69b8fab3865db32b7df554153ff
	Nivel 1 (desde la raíz):
		a21abd09aaa11e0e32a209daa7fbd54f5f4ca24838167a58f49972295a04623b
		73854883a00e4bfde544c29fc1f78c139f80074c109e307497db509c6cc4e5ee
	Nivel 2 (desde la raíz):
		eaf25202eacc17b609e7afb0e687994b8aec4e7be2f5c3fe1133145cffe20978
		f83509de6ace20c946229280efa601ead5e6dfb70742da9aef29bf0c5494f2e5
		2b45b7265fd7fff1021323d13d64f93e6cdf78d7b04a6aa05aaf0eae360bc40b
		9f8846814eea1c3f9ea4037b3bcc4005c6cc6b6a57e114a360ca9b7bad97ae63


Al crear el árbol podemos obtener la raíz del árbol y una lista con todos los niveles del árbol.

In [24]:
root, niveles = arbol_merkle.construir_merkle_tree()


Mostramos el hash del nodo raíz.

In [25]:
print(root)

6d84b95380981201807f01b53207747bb2d1e69b8fab3865db32b7df554153ff


Mostramos la lista de niveles.

In [26]:
print(niveles)

[['eaf25202eacc17b609e7afb0e687994b8aec4e7be2f5c3fe1133145cffe20978', 'f83509de6ace20c946229280efa601ead5e6dfb70742da9aef29bf0c5494f2e5', '2b45b7265fd7fff1021323d13d64f93e6cdf78d7b04a6aa05aaf0eae360bc40b', '9f8846814eea1c3f9ea4037b3bcc4005c6cc6b6a57e114a360ca9b7bad97ae63'], ['a21abd09aaa11e0e32a209daa7fbd54f5f4ca24838167a58f49972295a04623b', '73854883a00e4bfde544c29fc1f78c139f80074c109e307497db509c6cc4e5ee'], ['6d84b95380981201807f01b53207747bb2d1e69b8fab3865db32b7df554153ff']]


# Bloque

Un bloque es la unidad básica de estructura de una blockchain. Funciona como un eslabón de la cadena que almacena datos y se conecta criptográficamente con los bloques anteriores y posteriores, formando una cadena inmutable.

![](https://raw.githubusercontent.com/josejuansanchez/seminario-blockchain/refs/heads/main/images/blockchain_block_structure.png?token=GHSAT0AAAAAACZD4P4FUVL3URHAYWYSQ4K42BCUM7A)

En la blockchain que vamos a utilizar en este seminario, se ha diseñado un bloque básico que contiene los siguientes datos:

- **Hash del bloque:** Identificador único del bloque, generado a partir de su contenido. Se calcula mediante consenso.
- **Hash del bloque anterior:** Identificador único del bloque anterior, asegurando la integridad de la cadena.
- **Marca de tiempo:** Fecha y hora en que se creó el bloque.
- **Lista de transacciones:** Lista de transacciones incluidas en el bloque.
- **Árbol de Merkle:** Estructura de árbol que organiza las transacciones en pares cifrados. Nos permite verificar de forma sencilla si una transacción está incluida en el bloque.
- **Merkle Root:** Hash de la raíz del árbol de Merkle.
- **Nonce:** Número aleatorio utilizado en el proceso de minería (Proof of Work) para encontrar un hash válido.

In [27]:
class Bloque:
    def __init__(self, hash_anterior: str, l_transacciones: List[Transaccion], marca_de_tiempo: str):
        self.hash_anterior = hash_anterior
        self.marca_de_tiempo = marca_de_tiempo
        self.l_transacciones = l_transacciones
        self.merkle_tree = MerkleTree(l_transacciones)
        self.merkle_root = self.merkle_tree.root
        self.merkle_niveles = self.merkle_tree.niveles  # Guardamos todos los niveles del árbol
        self.nonce = 0
        self.hash = self.proof_of_work() # El cálculo de hash válido es necesario, pero no sería necesario almacenarlo

    # Calculamos el hash de los siguientes atributos del bloque:
    # - hash_anterior, marca_de_tiempo, merkle_root y nonce
    def calculate_hash(self) -> str:
        block_string = f"{self.hash_anterior}{self.marca_de_tiempo}{self.merkle_root}{self.nonce}"
        return hashlib.sha256(block_string.encode()).hexdigest()

    # Método que realiza la PoW (Proof of Work)
    # Busca el número mágico (nonce) para conseguir un hash con el número inicial de 0s requerido.
    # Es como la lotería. El que encuentre este número, gana y puede añadir el bloque.
    # Con esto dificultamos el añadir bloques a un asaltante.
    # Cuanto mayor número de 0s, más dificultad y más tiempo.
    def proof_of_work(self) -> str:
        while True:
            hash_result = self.calculate_hash()
            if hash_result.startswith("000"):
                return hash_result
            self.nonce += 1

    def __str__(self) -> str:
        # Creamos una cadena con los niveles del árbol de hashes en el bloque
        # La función reversed devuelve la lista en orden inverso sin modificar la lista original.
        # La función enumerate nos permite iterar sobre la lista devolviendo el índice y el valor que hay en esa posición.
        merkle_tree_str = "\nNiveles del Merckle Tree:\n"
        for i, level in enumerate(reversed(self.merkle_niveles)):
            merkle_tree_str += f"  Level {i}:\n"
            for hash_val in level:
                merkle_tree_str += f"    {hash_val}\n"

        return (
            f"Bloque--------------------------------------------------------------------------------\n"
            f"Hash previo: {self.hash_anterior}\n"
            f"Marca de tiempo: {self.marca_de_tiempo}\n"
            f"Raiz del Merkle Tree: {self.merkle_root}\n"
            f"Nonce: {self.nonce}\n"
            f"Hash: {self.hash}\n" # Guardamos el hash, aunque no sería necesario
            f"Listado de transacciones:\n" +
            "\n".join([f"  {tx}" for tx in self.l_transacciones]) + "\n" +
            merkle_tree_str + "\n" + f"Fin de bloque--------------------------------------------------------\n"

        )


Creamos un bloque génesis (sin transacciones) de prueba.

In [33]:
import time

bloque_genesis = Bloque("00000000000000", [], time.strftime("%Y-%m-%d %H:%M:%S"))

# Buscamos un hash que empiece con el número de 0s que le hemos indicado
bloque_genesis.proof_of_work()

'000ca25f4e94e53136fa28b9971c433fc7ed0f7a551a2adc4e792a10ff6620ec'

Creamos un bloque con las transacciones que creamos en los pasos anteriores.

In [40]:
bloque = Bloque(bloque_genesis.hash, lista_transacciones, time.strftime("%Y-%m-%d %H:%M:%S"))

Imprimimos el contenido del bloque que acabamos de crear.

In [42]:
print(bloque)

Bloque--------------------------------------------------------------------------------
Hash previo: 000ca25f4e94e53136fa28b9971c433fc7ed0f7a551a2adc4e792a10ff6620ec
Marca de tiempo: 2025-05-13 00:49:49
Raiz del Merkle Tree: 6d84b95380981201807f01b53207747bb2d1e69b8fab3865db32b7df554153ff
Nonce: 1062
Hash: 00048d4afeb97cb99d813c7167cc1f8bc1b5673e723711fbaba2080eba5edbc1
Listado de transacciones:
  Pepe envia a Juan la cantidad de 10
  María envia a Lucía la cantidad de 20
  Antonio envia a Ramón la cantidad de 30
  Carmen envia a Marta la cantidad de 40

Niveles del Merckle Tree:
  Level 0:
    6d84b95380981201807f01b53207747bb2d1e69b8fab3865db32b7df554153ff
  Level 1:
    a21abd09aaa11e0e32a209daa7fbd54f5f4ca24838167a58f49972295a04623b
    73854883a00e4bfde544c29fc1f78c139f80074c109e307497db509c6cc4e5ee
  Level 2:
    eaf25202eacc17b609e7afb0e687994b8aec4e7be2f5c3fe1133145cffe20978
    f83509de6ace20c946229280efa601ead5e6dfb70742da9aef29bf0c5494f2e5
    2b45b7265fd7fff1021323d13d64f93e

# Blockchain con Árbol de Merkle para almacenar las transacciones

Ahora vamos a unir todo el código que hemos visto en los pasos anteriores para crear una blockchain que hace uso de un árbol de Merkle para almacenar las transacciones.

In [43]:
import hashlib
import time
from typing import List, Dict

# Clase para almacenar los datos de una transacción
class Transaccion:
    def __init__(self, pagador: str, receptor: str, cantidad: float):
        self.pagador = pagador
        self.receptor = receptor
        self.cantidad = cantidad

    # Devuelve los datos de la transacción en un diccionario
    def convert_to_dict(self) -> Dict:
        return {
            'pagador': self.pagador,
            'receptor': self.receptor,
            'cantidad': self.cantidad
        }

    # Convertimos a un string el contenido de la transacción
    def __str__(self) -> str:
        return f"{self.pagador} envia a {self.receptor} la cantidad de {self.cantidad}"

# Clase para implementar un árbol de Merkle que almacena las transacciones de un bloque.
# Mirar los apuntes de EDA.
class MerkleTree:
    def __init__(self, l_transacciones: List[Transaccion]):
        self.l_transacciones = l_transacciones
        self.root, self.niveles = self.construir_merkle_tree()

    def construir_merkle_tree(self) -> tuple[str, List[List[str]]]:
        if not self.l_transacciones:
            return "", []

        # Generamos los hashs de la lista de transacciones (Versión Iterativa)
        hojas = []
        for tx in self.l_transacciones:
            # (1) Convertimos la transacción en un diccionario: tx.convert_to_dict()
            # (2) Convertimos el diccionario en una cadena: str(tx.convert_to_dict())
            # (3) Convertimos la cadena a bytes: str(tx.to_dict()).encode()
            # (4) Creamos el hash de la transacción y obtenemos su resumen en hexadecimal: hashlib.sha256(...).hexdigest()
            hash_transaccion = hashlib.sha256(str(tx.convert_to_dict()).encode()).hexdigest()

            # Añadimos el hash a la lista de hojas
            hojas.append(hash_transaccion)

        # Guardamos los niveles del árbol de Merkle
        niveles = [hojas]

        # Si son más transacciones, calculamos los hash de cada una de ellas y construimos el árbol
        # Construimos el árbol, de abajo a arriba
        # Acordaos de lo que habéis visto en EDA
        nivel_actual = hojas
        while len(nivel_actual) > 1:
            nivel_temp = []
            for i in range(0, len(nivel_actual), 2):
                if i + 1 < len(nivel_actual):
                    # Juntamos dos hojas
                    combined = nivel_actual[i] + nivel_actual[i + 1]
                    nuevo_hash = hashlib.sha256(combined.encode()).hexdigest()
                    nivel_temp.append(nuevo_hash)
                else:
                    # Hoja impar primero
                    nivel_temp.append(nivel_actual[i])
            niveles.append(nivel_temp)
            nivel_actual = nivel_temp

        return nivel_actual[0], niveles

    def mostrar_niveles(self) -> None:
        print("\nNiveles del Árbol de Merkle:")
        for idx, nivel in enumerate(reversed(self.niveles)):
            print(f"\tNivel {idx} (desde la raíz):")
            for hash in nivel:
                print(f"\t\t{hash}")

# Clase para almacenar un bloque de la blockchain.
# Incluye toda la información que se ha estudiado en el tema 2.
class Bloque:
    def __init__(self, hash_anterior: str, l_transacciones: List[Transaccion], marca_de_tiempo: str):
        self.hash_anterior = hash_anterior
        self.marca_de_tiempo = marca_de_tiempo
        self.l_transacciones = l_transacciones
        self.merkle_tree = MerkleTree(l_transacciones)
        self.merkle_root = self.merkle_tree.root
        self.merkle_niveles = self.merkle_tree.niveles  # Guardamos todos los niveles del árbol
        self.nonce = 0
        self.hash = self.proof_of_work() # El cálculo de hash válido es necesario, pero no sería necesario almacenarlo

    # Calculamos el hash de los siguientes atributos del bloque:
    # - hash_anterior, marca_de_tiempo, merkle_root y nonce
    def calculate_hash(self) -> str:
        block_string = f"{self.hash_anterior}{self.marca_de_tiempo}{self.merkle_root}{self.nonce}"
        return hashlib.sha256(block_string.encode()).hexdigest()

    # Método que realiza la PoW (Proof of Work)
    # Busca el número mágico (nonce) para conseguir un hash con el número inicial de 0s requerido.
    # Es como la lotería. El que encuentre este número, gana y puede añadir el bloque.
    # Con esto dificultamos el añadir bloques a un asaltante.
    # Cuanto mayor número de 0s, más dificultad y más tiempo.
    def proof_of_work(self) -> str:
        while True:
            hash_result = self.calculate_hash()
            if hash_result.startswith("000"):
                return hash_result
            self.nonce += 1

    def __str__(self) -> str:
        # Creamos una cadena con los niveles del árbol de hashes en el bloque
        # La función reversed devuelve la lista en orden inverso sin modificar la lista original.
        # La función enumerate nos permite iterar sobre la lista devolviendo el índice y el valor que hay en esa posición.
        merkle_tree_str = "\nNiveles del Merckle Tree:\n"
        for i, level in enumerate(reversed(self.merkle_niveles)):
            merkle_tree_str += f"  Level {i}:\n"
            for hash_val in level:
                merkle_tree_str += f"    {hash_val}\n"

        return (
            f"Bloque--------------------------------------------------------------------------------\n"
            f"Hash previo: {self.hash_anterior}\n"
            f"Marca de tiempo: {self.marca_de_tiempo}\n"
            f"Raiz del Merkle Tree: {self.merkle_root}\n"
            f"Nonce: {self.nonce}\n"
            f"Hash: {self.hash}\n" # Guardamos el hash, aunque no sería necesario
            f"Listado de transacciones:\n" +
            "\n".join([f"  {tx}" for tx in self.l_transacciones]) + "\n" +
            merkle_tree_str + "\n" + f"Fin de bloque--------------------------------------------------------\n"

        )

# Clase para implementar la blockchain
class Blockchain:
    def __init__(self):
        # Una lista enlazada
        self.chain = [self.crea_bloque_genesis()]

    def crea_bloque_genesis(self) -> Bloque:
        return Bloque("00000000000000", [], time.strftime("%Y-%m-%d %H:%M:%S"))

    def obtener_ultimo_bloque(self) -> Bloque:
        return self.chain[-1]

    def append_bloque(self, l_transacciones: List[Transaccion]):
        nuevo_bloque = Bloque(
            hash_anterior = self.obtener_ultimo_bloque().hash,
            l_transacciones = l_transacciones,
            marca_de_tiempo = time.strftime("%Y-%m-%d %H:%M:%S")
        )

        self.chain.append(nuevo_bloque)

        # Print Merkle Tree for the new block
        print("\nÁrbol de Merkle del nuevo bloque:")
        for i, nivel in enumerate(reversed(nuevo_bloque.merkle_niveles)):
            print(f"Nivel {i}:")
            for valor_de_hash in nivel:
                print(f"  {valor_de_hash}")
        print(f"Raíz de Merkle: {nuevo_bloque.merkle_root}\n")

    def __str__(self) -> str:
        return "\n".join([str(bloque) for bloque in self.chain])

# Programa principal
def main():
    blockchain = Blockchain()

    while True:
        print("\n=== Añadir nuevo bloque ===")
        while True:
                try:
                    num_transacciones = abs(int(input("Número de transacciones en el bloque: ")))
                    break
                except ValueError:
                    print("Por favor, introduce un número válido.")

        print(f"Se generarán {num_transacciones} transacciones para este bloque.")

        l_transacciones = []
        for i in range(num_transacciones):
            print(f"\nTransacción {i + 1}:")
            pagador = input("Remitente: ")
            receptor = input("Destinatario: ")
            while True:
                try:
                    cantidad = float(input("Cantidad: "))
                    break
                except ValueError:
                    print("Por favor, introduce un número válido.")

            l_transacciones.append(Transaccion(pagador, receptor, cantidad))

        blockchain.append_bloque(l_transacciones)
        print("\n=== Blockchain Actualizada ===")
        print(blockchain)

        eleccion = input("\n¿Desea añadir otro bloque? (s/n): ").lower()
        if eleccion != 's':
            break

if __name__ == "__main__":
    main()


=== Añadir nuevo bloque ===
Número de transacciones en el bloque: 5
Se generarán 5 transacciones para este bloque.

Transacción 1:
Remitente: Juan
Destinatario: Pepe
Cantidad: 10

Transacción 2:
Remitente: María
Destinatario: Carmen
Cantidad: 20

Transacción 3:
Remitente: Alfredo
Destinatario: Manolo
Cantidad: 30

Transacción 4:
Remitente: Carmen
Destinatario: Valentina
Cantidad: 40

Transacción 5:
Remitente: Manolo
Destinatario: Paco
Cantidad: 50

Árbol de Merkle del nuevo bloque:
Nivel 0:
  247fedc60a70b11e48a2f44a33e21fba4d12b181d7c4139a9fdeb1507287b0d1
Nivel 1:
  4e82005fe2b10f67b5a0a639894b244c797b7765b04ce9d159786d3799cb9136
  dc94950f8980a9379838e2a2b3fd0ab20ca46ee18e0d82c9bcf8d578029832da
Nivel 2:
  0daaa13f7f8630728d91e8ed0831d9efc588966760fb30407c95268c1dc225c7
  233cac298957312c809306cbe4ac130d10e8048d6621d92a93a9da405863e42a
  dc94950f8980a9379838e2a2b3fd0ab20ca46ee18e0d82c9bcf8d578029832da
Nivel 3:
  7f9a83396dbde451a487efffe7d49d839b542cbcae283a19719ef2f603026b7f
  b4cdc